# Check Exposure list from Butler (saved in csv file)

- author : Sylvie Dagoret-Campagne
- creation date : 2025-09-15
- last update : 2025-09-16
- last update : 2025-09-18 : check both repos
- last update : 2025-09-19 : add Target
- last update : 2025-09-20 : Chage path of inputs

- read the list of exposure generated by TOOL_ListofExposures.ipynb
- plot the different filters vs time

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os

plt.rcParams["figure.figsize"] = (16,8)
plt.rcParams["axes.labelsize"] = 'xx-large'
plt.rcParams['axes.titlesize'] = 'xx-large'
plt.rcParams['xtick.labelsize']= 'xx-large'
plt.rcParams['ytick.labelsize']= 'xx-large'
plt.rcParams["legend.fontsize"] = "xx-large"



In [ ]:
FLAG_REPO_EMBARGO = True
if FLAG_REPO_EMBARGO:
    repo="/repo/embargo"
else:
    repo="/repo/main"
reponame = repo.replace("/","_")

path_exposureslist = "data/butlerregistry"
listexposures_file = os.path.join(path_exposureslist,"holosummary_all_filters" + reponame + ".csv")

In [ ]:
df = pd.read_csv(listexposures_file,index_col=0)
df = df.reset_index(drop=True)

In [ ]:
df

In [ ]:
# conversion en datetime
df["time_start"] = pd.to_datetime(df["time_start"])
df["time_end"]   = pd.to_datetime(df["time_end"])

In [ ]:
# Remove to run faster the notebook
#import ipywidgets as widgets
#%matplotlib widget

## dependance with filter time

In [ ]:
plt.figure(figsize=(20,8))

# conversion en datetime
df["date"] = pd.to_datetime(df["day_obs"].astype(str), format="%Y%m%d")


sns.scatterplot(
    data=df, 
    x="date",       # abscisse en datetime
    y="seq_num",    # ou ra, dec, etc.
    hue="filter", 
    palette="tab10"
)

plt.title(f"Auxtel Holo observations wrt date and filter type ({reponame})")
plt.xlabel("Date of observation")
plt.ylabel("Seq Num")
plt.xticks(rotation=45)  # lisibilité des dates
#plt.legend(loc="upper left",ncol=8)
plt.legend(bbox_to_anchor=(1.01, 1.05),ncols=1)
plt.grid()
plt.tight_layout()
plt.show()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns



plt.figure(figsize=(20,8))
sns.scatterplot(
    data=df,
    x="time_start",   # abscisse
    y="seq_num",      # tu peux mettre ra/dec si tu préfères
    hue="filter",
    palette="tab10"
)

plt.title(f"Auxtel Holo observations wrt date and filter type ({reponame})")
plt.xlabel("Time")
plt.ylabel("Seq Num")
plt.xticks(rotation=45)
#plt.legend(loc="upper left",ncol=8)
plt.legend(bbox_to_anchor=(1.01, 1.05),ncols=1)
plt.grid()
plt.tight_layout()
plt.show()


In [ ]:
plt.figure(figsize=(20,8))
sns.scatterplot(
    data=df,
    x="time_start",
    y="seq_num",
    hue="filter",
    palette="Set2",       # palette plus contrastée
    s=30,                # taille des points
    edgecolor="black",    # contour noir
    linewidth=0.2
)

plt.title(f"Auxtel Holo observations wrt date and filter type ({reponame})")
plt.xlabel("Time")
plt.ylabel("Seq Num")
plt.xticks(rotation=45)
#plt.legend(loc="upper left", ncol=8)
plt.legend(bbox_to_anchor=(1.01, 1.05),ncols=1)
plt.tight_layout()
plt.grid()
plt.show()


In [ ]:
plt.figure(figsize=(20,8))
sns.scatterplot(
    data=df,
    x="time_start",
    y="filter",    # chaque catégorie a une ligne
    hue="filter",
    palette="Set1",
    s=100,
    edgecolor="black",
    linewidth=0.2
)

plt.title(f"Auxtel Holo observations wrt date and filter type ({reponame})")
plt.xlabel("Time")
plt.ylabel("Filter")
plt.xticks(rotation=45)
plt.tight_layout()
#plt.legend(ncols=5)
plt.legend(bbox_to_anchor=(1.01, 1.05),ncols=1)
plt.grid()
plt.show()


In [ ]:
import numpy as np

# on décale légèrement la position en y en fonction de seq_num
df["filter_seq"] = df["filter"].astype(str) + "_" + df["seq_num"].astype(str)

plt.figure(figsize=(20,8))
sns.stripplot(
    data=df,
    x="time_start",
    y="filter",
    hue="filter",
    palette="Set1",
    size=4,         # taille des points
    jitter=True,    # évite que les points se chevauchent
    alpha=1.0
)

plt.title(f"Auxtel Holo observations wrt date and filter type ({reponame})",fontsize=15)
plt.xlabel("Time")
plt.ylabel("Filter")
plt.xticks(rotation=45)
#plt.legend(loc="upper left", ncol=6)
plt.grid()
plt.tight_layout()
plt.show()


In [ ]:
plt.figure(figsize=(20,16))
df["target_seq"] = df["target"].astype(str) + "_" + df["seq_num"].astype(str)
sns.stripplot(
    data=df,
    x="time_start",
    y="target",
    hue="target",
    palette="Set2",
    size=10,         # taille des points
    jitter=True,    # évite que les points se chevauchent
    alpha=1.0,
    edgecolor="black",
    linewidth=0.1
)
plt.title(f"Auxtel Holo observations wrt date and target ({reponame})",fontsize=15)
plt.xlabel("Time")
plt.ylabel("Target")
plt.xticks(rotation=45)
plt.tight_layout()
plt.grid()
plt.show()